### Scraping fighters data

In [2]:
import requests,bs4
import helpers

In [3]:
BASE_URL = "http://ufcstats.com/statistics/fighters"

In [4]:
import string

# The page is organized by fighters names so I decided to
# Loop through different pages using an alphabets list
alphabets = list(string.ascii_lowercase)
for letter in alphabets:
    fighters_DOM_per_letter = helpers.cached_request(f"{BASE_URL}?char={letter}")
    soup = bs4.BeautifulSoup(fighters_DOM_per_letter, 'html.parser')
    pagination = soup.find('ul', class_='b-statistics__paginate')
    n_pages = len(pagination.find_all('li'))

    # Initialize the fighters dict
    if letter == 'a':
        headers = [th.get_text(strip=True) for th in soup.select("table thead th")]
        fighters_data = {header.title(): [] for header in headers}
    
    # Not adding 1 to the number of list items because
    # there is a link for "all"
    for i in range(1, n_pages):
        current_page = helpers.cached_request(f"{BASE_URL}?char={letter}&page={i}")
        soup_1 = bs4.BeautifulSoup(current_page, 'html.parser')

        # Scraping fighters tabular data 
        for row in soup_1.select("table tbody tr"):
            cells = row.find_all('td',class_='b-statistics__table-col')
            
            if len(cells) == 0 :
                continue
            
            while len(cells) < len(headers):
                cells.append(None)
            
            for header,cell in zip(headers,cells):
                header = header.title()
                if cell:
                    fighters_data[header].append(cell.get_text(strip=True))
                else :
                    fighters_data[header].append(None)

In [5]:
import pandas as pd

df = pd.DataFrame(fighters_data)

In [6]:
df.tail()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
3562,Zhang,Weili,Magnum,"5' 4""",115 lbs.,"63.0""",Switch,26,3,0,
3563,Zhang,Mingyang,Mountain Tiger,"6' 2""",205 lbs.,"75.0""",Orthodox,19,7,0,
3564,Zhang,Qinghe,,"5' 7""",135 lbs.,"67.0""",Southpaw,16,8,1,
3565,Zhang,Zhenyang,,--,155 lbs.,--,,7,2,0,
3566,Daermisi,Zhawupasi,,"5' 8""",135 lbs.,"68.0""",Switch,9,2,0,


### Scraping Events Data

In [7]:
BASE_URL = "http://ufcstats.com/statistics/events/completed?page=all"

In [8]:
events_html = helpers.cached_request("http://ufcstats.com/statistics/events/completed?page=all")

In [9]:
events_soup = bs4.BeautifulSoup(events_html,'html.parser')

# Get table headers
events_table_headers = [th.get_text(strip=True) for th in events_soup.select('table thead th')]
# The first header is Name/Date so it is better to split it into separate name & date
events_table_headers = events_table_headers[0].split('/')+events_table_headers[1:]
events_dict = {header.title() :[] for header in events_table_headers}
events_rows = events_soup.select('table tbody tr.b-statistics__table-row')
# The first element is an empty row  
events_rows.pop(0)
for row in events_rows:
    
    first_td = row.select_one('td.b-statistics__table-col')
    event_name = first_td.select_one('a.b-link').get_text(strip=True)
    event_date = first_td.select_one('span.b-statistics__date').get_text(strip=True)

    location = row.select_one('td.b-statistics__table-col_style_big-top-padding').get_text(strip=True)

    events_info = [event_name,event_date,location]
    
    for header,cell in zip(events_table_headers,events_info):
        header = header.title()
        events_dict[header].append(cell)
        

In [10]:
# Storing events in a dataframe 
events_df = pd.DataFrame(events_dict)
events_df.head()

,Name,Date,Location
0,UFC Fight Night: Walker vs. Zhang,"August 23, 2025","Shanghai, Hebei, China"
1,UFC 319: Du Plessis vs. Chimaev,"August 16, 2025","Chicago, Illinois, USA"
2,UFC Fight Night: Dolidze vs. Hernandez,"August 09, 2025","Las Vegas, Nevada, USA"
3,UFC Fight Night: Taira vs. Park,"August 02, 2025","Las Vegas, Nevada, USA"
4,UFC Fight Night: Whittaker vs. De Ridder,"July 26, 2025","Abu Dhabi, Abu Dhabi, United Arab Emirates"


### Scraping Fights Data

In [ ]:
# I already have events_rows
first_row = True
# Loop through them
for row in events_rows:
    first_td = row.select_one('td.b-statistics__table-col')
    event_link = first_td.select_one('a.b-link').get('href')
    # Visit each event link
    event_html = helpers.cached_request(event_link)
    event_soup = bs4.BeautifulSoup(event_html, "html.parser")
    if first_row:
        fights_headers = ["Win/No Contest/Draw",
                          "Fighter_1",
                          "Fighter_2",
                          "KD_1",
                          "KD_2",
                          "STR_1",
                          "STR_2",
                          "TD_1",
                          "TD_2",
                          "SUB_1",
                          "SUB_2",
                          "Weight_Class",
                          "Method",
                          "Round",
                          "Fight_Time"
                          ]
        fights_dict = {header: [] for header in fights_headers}
        first_row = False
        
    # For each event, fights are arranged in tables
    fight_rows = event_soup.select(
        "table.b-fight-details__table tbody tr.b-fight-details__table-row")
    for fight_row in fight_rows:
        cells = fight_row.select('td')

        # Get fights data
        result_flag = cells[0].get_text(strip=True)

        fighters = [a.get_text(strip=True)
                    for a in cells[1].select("a.b-link")]
        kd = [p.get_text(strip=True) for p in cells[2].select("p")]
        strikes = [p.get_text(strip=True) for p in cells[3].select("p")]
        td = [p.get_text(strip=True) for p in cells[4].select("p")]
        sub = [p.get_text(strip=True) for p in cells[5].select("p")]
        weight_class = cells[6].get_text(strip=True)
        method = " ".join([p.get_text(strip=True)
                          for p in cells[7].select("p") if p.get_text(strip=True)])
        round_num = cells[8].get_text(strip=True)
        fight_time = cells[9].get_text(strip=True)

        fight_info = [result_flag, *fighters, *kd, *strikes, *
                      td, *sub, weight_class, method, round_num, fight_time]

        for col, val in zip(fights_dict.values(), fight_info):
            col.append(val)

In [28]:
fights_df = pd.DataFrame(fights_dict)
fights_df['Weight_Class'].value_counts()

Weight_Class
Lightweight              1409
Welterweight             1351
Middleweight             1101
Featherweight             817
Heavyweight               741
Bantamweight              739
Light Heavyweight         723
Flyweight                 387
Women's Strawweight       346
Women's Flyweight         260
Women's Bantamweight      229
Open Weight               117
Catch Weight               73
Women's Featherweight      30
Super Heavyweight           1
Name: count, dtype: int64